In [2]:
import pandas as pd
import numpy as np
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import collections
import itertools

import scipy.stats as stats
from scipy.stats import norm
from scipy.special import boxcox1p

import statsmodels
import statsmodels.api as sm


from sklearn.preprocessing import scale, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ElasticNet,  HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.utils import resample
from xgboost import XGBRegressor



import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
final_df = pd.read_excel('/content/model.xlsx')

In [5]:
categorical_features = final_df.select_dtypes(include=['object'])
print('Categorical features: {}'.format(categorical_features.shape))

Categorical features: (48970, 4)


In [6]:
categorical_features_one_hot = pd.get_dummies(categorical_features)
categorical_features_one_hot.head()

,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_No Neighbourhood group,neighbourhood_group_Queens,neighbourhood_group_Staten Island,neighbourhood_Allerton,neighbourhood_Arden Heights,neighbourhood_Arrochar,neighbourhood_Arverne,...,"neighbourhood_Woodridge, Fort Lincoln, Gateway",neighbourhood_Woodrow,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room,location_New Jersey,location_New york,location_Washington D.C
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [7]:
final_df['reviews_per_month'] = final_df['reviews_per_month'].fillna(0)

In [8]:
numerical_features =  final_df.select_dtypes(exclude=['object'])
y = numerical_features.price
numerical_features = numerical_features.drop(['price'], axis=1)
print('Numerical features: {}'.format(numerical_features.shape))

Numerical features: (48970, 12)


In [9]:
numerical_features.isna().sum()

latitude                          0
longitude                         0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
all_year_avail                    0
low_avail                         0
no_reviews                        0
dtype: int64

In [10]:
X = np.concatenate((numerical_features, categorical_features_one_hot), axis=1)
X_df = pd.concat([numerical_features, categorical_features_one_hot], axis=1)

In [11]:
Processed_data = pd.concat([X_df, y], axis = 1)

In [12]:
y.head(1)

0    5.620401
Name: price, dtype: float64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
print('Dimensions of the training feature matrix: {}'.format(X_train.shape))
print('Dimensions of the training target vector: {}'.format(y_train.shape))
print('Dimensions of the test feature matrix: {}'.format(X_test.shape))
print('Dimensions of the test target vector: {}'.format(y_test.shape))

Dimensions of the training feature matrix: (34279, 292)
Dimensions of the training target vector: (34279,)
Dimensions of the test feature matrix: (14691, 292)
Dimensions of the test target vector: (14691,)


In [15]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [16]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor()

# train the regressor on the training set
regressor.fit(X_train, y_train)

# make predictions on the testing set
y_pred = regressor.predict(X_test)

# calculate the mean squared error of the regressor
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
r2 = r2_score(y_test, y_pred)

print("R2 Score:", r2)

Mean Squared Error: 0.4964826066055863
R2 Score: 0.20455694440763628


In [17]:
from sklearn import datasets
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# train the regressor on the training set
regressor.fit(X_train, y_train)

# make predictions on the testing set
y_pred = regressor.predict(X_test)

# calculate the mean squared error and R2 score of the regressor
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 0.2450401177580865
R2 Score: 0.6074072738522964


In [18]:
import pickle
with open('regressor.pickle', 'wb') as f:
    pickle.dump(regressor, f)

In [135]:
X_df.head(1)['low_avail']

0    False
Name: low_avail, dtype: bool

In [19]:

c = list(X_df.head(1).columns)

In [43]:
c[12:18]

['neighbourhood_group_Bronx',
 'neighbourhood_group_Brooklyn',
 'neighbourhood_group_Manhattan',
 'neighbourhood_group_No Neighbourhood group',
 'neighbourhood_group_Queens',
 'neighbourhood_group_Staten Island']

In [45]:
c[18:-7]

['neighbourhood_Allerton',
 'neighbourhood_Arden Heights',
 'neighbourhood_Arrochar',
 'neighbourhood_Arverne',
 'neighbourhood_Astoria',
 'neighbourhood_Bath Beach',
 'neighbourhood_Battery Park City',
 'neighbourhood_Bay Ridge',
 'neighbourhood_Bay Terrace',
 'neighbourhood_Baychester',
 'neighbourhood_Bayside',
 'neighbourhood_Bayswater',
 'neighbourhood_Bedford-Stuyvesant',
 'neighbourhood_Belle Harbor',
 'neighbourhood_Bellerose',
 'neighbourhood_Belmont',
 'neighbourhood_Bensonhurst',
 'neighbourhood_Bergen Beach',
 'neighbourhood_Boerum Hill',
 'neighbourhood_Borough Park',
 'neighbourhood_Breezy Point',
 'neighbourhood_Briarwood',
 'neighbourhood_Brighton Beach',
 'neighbourhood_Brightwood Park, Crestwood, Petworth',
 'neighbourhood_Bronxdale',
 'neighbourhood_Brookland, Brentwood, Langdon',
 'neighbourhood_Brooklyn Heights',
 'neighbourhood_Brownsville',
 "neighbourhood_Bull's Head",
 'neighbourhood_Bushwick',
 'neighbourhood_Cambria Heights',
 'neighbourhood_Canarsie',
 'neig

In [47]:
c[-7:-3]

['room_type_Entire home/apt',
 'room_type_Hotel room',
 'room_type_Private room',
 'room_type_Shared room']

In [48]:
c[-3:]

['location_New Jersey', 'location_New york', 'location_Washington D.C']

In [18]:
ng = c[12:18]

In [22]:
ng = c[12:18]
ne = c[18:-7]
rt = c[-7:-3]
loc = c[-3:]
loc = [s.replace(" ", "_") for s in loc]
ng = [s.replace(" ", "_") for s in ng]
ne = [s.replace(" ", "_") for s in ne]
locd =  {var: 0 for var in loc}
ngd =  {var: 0 for var in ng}
ned =  {var: 0 for var in ne}

In [23]:
ne = c[18:-7]

In [72]:
len(ne)

267

In [73]:
len(ng)

6

In [74]:
len(rt)

4

In [24]:
rt = c[-7:-3]

In [25]:
rt

['room_type_Entire home/apt',
 'room_type_Hotel room',
 'room_type_Private room',
 'room_type_Shared room']

In [26]:
loc = c[-3:]

In [27]:
loc

['location_New Jersey', 'location_New york', 'location_Washington D.C']

In [28]:
loc = [s.replace(" ", "_") for s in loc]

In [29]:
ng = [s.replace(" ", "_") for s in ng]
ne = [s.replace(" ", "_") for s in ne]

In [30]:
locd =  {var: 0 for var in loc}

In [31]:
ngd =  {var: 0 for var in ng}
ned =  {var: 0 for var in ne}

In [32]:
a = []
a = X_df.head(1)
a = X_df.head(1).values.tolist()
print(a)

[[40.66265, -73.99454, 3.091042453358316, 3, 4108, 0.03, 1, 267, 1, False, False, False, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]]


In [33]:
import pickle
import math

# load the saved model
with open('regressor.pickle', 'rb') as f:
    regressor = pickle.load(f)

# make predictions on new data
new_data = a
prediction = regressor.predict(new_data)

print(math.exp(prediction))

183.98946413687293


In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, None],
    'max_features': ['sqrt', 'log2', None]
}

# Create a random forest regressor
regressor = RandomForestRegressor(random_state=42)

# Create a grid search object
grid_search = GridSearchCV(
    estimator=regressor,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1
)

# Fit the grid search object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Make predictions on the testing set using the best model
best_regressor = grid_search.best_estimator_
y_pred = best_regressor.predict(X_test)

# Calculate the R2 score of the best model
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)


KeyboardInterrupt: ignored

In [35]:
!pip install streamlit_folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.4/423.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=db7bf8bb218221d998982eeb6854f9992b41448eaa591484c28d05e9b94390d5
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [154]:
%%writefile app.py

import streamlit as st
import pickle
import math

import pandas as pd


import folium

import streamlit as st
from folium.plugins import Draw

from streamlit_folium import st_folium

final_df = pd.read_excel('/content/model.xlsx')
categorical_features = final_df.select_dtypes(include=['object'])
categorical_features_one_hot = pd.get_dummies(categorical_features)
final_df['reviews_per_month'] = final_df['reviews_per_month'].fillna(0)
numerical_features =  final_df.select_dtypes(exclude=['object'])
y = numerical_features.price
numerical_features = numerical_features.drop(['price'], axis=1)
X_df = pd.concat([numerical_features, categorical_features_one_hot], axis=1)
c = list(X_df.head(1).columns)
ng = c[12:18]
ne = c[18:-7]
rt = c[-7:-3]
loc = c[-3:]
loc = [s.replace(" ", "_") for s in loc]
ng = [s.replace(" ", "_") for s in ng]
ne = [s.replace(" ", "_") for s in ne]




# load the saved model
with open('regressor.pickle', 'rb') as f:
    regressor = pickle.load(f)

# function to make predictions
def predict_price(latitude, longitude, minimum_nights, number_of_reviews,last_review, reviews_per_month, calculated_host_listings_count,availability_365, number_of_reviews_ltm, all_year_avail, low_avail,no_reviews,var_1, var_2, var_3, var_4, var_5, var_6, var_7, var_8, var_9, var_10, var_11, var_12, var_13, var_14, var_15, var_16, var_17, var_18, var_19, var_20, var_21, var_22, var_23, var_24, var_25, var_26, var_27, var_28, var_29, var_30, var_31, var_32, var_33, var_34, var_35, var_36, var_37, var_38, var_39, var_40, var_41, var_42, var_43, var_44, var_45, var_46, var_47, var_48, var_49, var_50, var_51, var_52, var_53, var_54, var_55, var_56, var_57, var_58, var_59, var_60, var_61, var_62, var_63, var_64, var_65, var_66, var_67, var_68, var_69, var_70, var_71, var_72, var_73, var_74, var_75, var_76, var_77, var_78, var_79, var_80, var_81, var_82, var_83, var_84, var_85, var_86, var_87, var_88, var_89, var_90, var_91, var_92, var_93, var_94, var_95, var_96, var_97, var_98, var_99, var_100, var_101, var_102, var_103, var_104, var_105, var_106, var_107, var_108, var_109, var_110, var_111, var_112, var_113, var_114, var_115, var_116, var_117, var_118, var_119, var_120, var_121, var_122, var_123, var_124, var_125, var_126, var_127, var_128, var_129, var_130, var_131, var_132, var_133, var_134, var_135, var_136, var_137, var_138, var_139, var_140, var_141, var_142, var_143, var_144, var_145, var_146, var_147, var_148, var_149, var_150, var_151, var_152, var_153, var_154, var_155, var_156, var_157, var_158, var_159, var_160, var_161, var_162, var_163, var_164, var_165, var_166, var_167, var_168, var_169, var_170, var_171, var_172, var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200,var_201,var_202,var_203,var_204,var_205,var_206,var_207,var_208,var_209,var_210,var_211,var_212,var_213,var_214,var_215,var_216,var_217,var_218,var_219,var_220,var_221,var_222,var_223,var_224,var_225,var_226,var_227,var_228,var_229,var_230,var_231,var_232,var_233,var_234,var_235,var_236,var_237,var_238,var_239,var_240,var_241,var_242,var_243,var_244,var_245,var_246,var_247,var_248,var_249,var_250,var_251,var_252,var_253,var_254,var_255,var_256,var_257,var_258,var_259,var_260,var_261,var_262,var_263,var_264,var_265,var_266,var_267,var_268,var_269,var_270,var_271,var_272,var_273,var_274,var_275,var_276,var_277,var_278,var_279,var_280):
  # create a list of input features
  new_data = [[latitude, longitude, minimum_nights, number_of_reviews,last_review, reviews_per_month, calculated_host_listings_count,availability_365, number_of_reviews_ltm, all_year_avail, low_avail,no_reviews,var_1, var_2, var_3, var_4, var_5, var_6, var_7, var_8, var_9, var_10, var_11, var_12, var_13, var_14, var_15, var_16, var_17, var_18, var_19, var_20, var_21, var_22, var_23, var_24, var_25, var_26, var_27, var_28, var_29, var_30, var_31, var_32, var_33, var_34, var_35, var_36, var_37, var_38, var_39, var_40, var_41, var_42, var_43, var_44, var_45, var_46, var_47, var_48, var_49, var_50, var_51, var_52, var_53, var_54, var_55, var_56, var_57, var_58, var_59, var_60, var_61, var_62, var_63, var_64, var_65, var_66, var_67, var_68, var_69, var_70, var_71, var_72, var_73, var_74, var_75, var_76, var_77, var_78, var_79, var_80, var_81, var_82, var_83, var_84, var_85, var_86, var_87, var_88, var_89, var_90, var_91, var_92, var_93, var_94, var_95, var_96, var_97, var_98, var_99, var_100, var_101, var_102, var_103, var_104, var_105, var_106, var_107, var_108, var_109, var_110, var_111, var_112, var_113, var_114, var_115, var_116, var_117, var_118, var_119, var_120, var_121, var_122, var_123, var_124, var_125, var_126, var_127, var_128, var_129, var_130, var_131, var_132, var_133, var_134, var_135, var_136, var_137, var_138, var_139, var_140, var_141, var_142, var_143, var_144, var_145, var_146, var_147, var_148, var_149, var_150, var_151, var_152, var_153, var_154, var_155, var_156, var_157, var_158, var_159, var_160, var_161, var_162, var_163, var_164, var_165, var_166, var_167, var_168, var_169, var_170, var_171, var_172, var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200,var_201,var_202,var_203,var_204,var_205,var_206,var_207,var_208,var_209,var_210,var_211,var_212,var_213,var_214,var_215,var_216,var_217,var_218,var_219,var_220,var_221,var_222,var_223,var_224,var_225,var_226,var_227,var_228,var_229,var_230,var_231,var_232,var_233,var_234,var_235,var_236,var_237,var_238,var_239,var_240,var_241,var_242,var_243,var_244,var_245,var_246,var_247,var_248,var_249,var_250,var_251,var_252,var_253,var_254,var_255,var_256,var_257,var_258,var_259,var_260,var_261,var_262,var_263,var_264,var_265,var_266,var_267,var_268,var_269,var_270,var_271,var_272,var_273,var_274,var_275,var_276,var_277,var_278,var_279,var_280]]
  # make predictions on new data
  prediction = regressor.predict(new_data)
  return math.exp(prediction)

# set app title and description
st.set_page_config(page_title="Airbnb Price Predictor", page_icon=":money_with_wings:")
st.title("Airbnb Price Predictor")
st.write("Enter the following information to get a price prediction for your Airbnb rental:")

ng = [s.replace("neighbourhood_group_", "") for s in ng]
ne = [s.replace("neighbourhood_", "") for s in ne]
rt = [s.replace("room_type_", "") for s in rt]
loc = [s.replace("location_", "") for s in loc]

ngd =  {var: 0 for var in ng}
ned =  {var: 0 for var in ne}
rtd =  {var: 0 for var in rt}
locd =  {var: 0 for var in loc}
location ,neighbor , negroup = st.columns(3)
with location:
  location = st.radio("Select a location:", ["New york", "Washington D.C", "New Jersey"])
with neighbor:
  neighbor = st.selectbox('Select Neighbourhood area:', ne)
with negroup:
  negroup = st.selectbox('Select an Neighbourbood group:', ng)

ngd[negroup] = 1
ned[neighbor] = 1
location = location.replace(" ", "_")
locd[location] = 1




selected_item = st.selectbox('Select an room type:', rt)
rtd[selected_item] = 1



if location == "New york":
    lat = 40.712776
    lon = -74.005974
elif location == "Washington D.C":
    lat = 38.9072
    lon = -77.0369
else:
    lat = 40.0583
    lon = -74.4057

m = folium.Map(location=[lat, lon], zoom_start=10)
Draw(export=False).add_to(m)

output = st_folium(m, width=1000, height=500)
st.write(output['all_drawings'])

if(output['all_drawings'] != None):
  st.write(output['all_drawings'][0]['geometry']['coordinates'][0])
  st.write(output['all_drawings'][0]['geometry']['coordinates'][1])
  latitude = output['all_drawings'][0]['geometry']['coordinates'][1]
  longitude = output['all_drawings'][0]['geometry']['coordinates'][1]

# input fields for other features
minimum_nights, number_of_reviews, last_review, reviews_per_month = st.columns(4)
with minimum_nights:
    minimum_nights = st.number_input("Minimum Nights", value=1, min_value=1, max_value=30, step=1)
with number_of_reviews:
    number_of_reviews = st.number_input("Number of Reviews", value=0, min_value=0, step=1)
with last_review:
    last_review = st.number_input("Last Review (in days)", value=0, min_value=0, step=1)
with reviews_per_month:
    reviews_per_month = st.number_input("Reviews per Month", value=0, min_value=0, step=1)


calculated_host_listings_count, availability_365, number_of_reviews_ltm = st.columns(3)
with calculated_host_listings_count:
    calculated_host_listings_count = st.number_input("Calculated Host Listings Count", value=1, min_value=1, step=1)
with availability_365:
    availability_365 = st.number_input("Availability (in days)", value=365, min_value=0, max_value=365, step=1)
with number_of_reviews_ltm:
    number_of_reviews_ltm = st.number_input("Number of Reviews Last 12 Months", value=0, min_value=0, step=1)



ngr = [s.replace("neighbourhood_group_", "") for s in c[12:18]]
neb = [s.replace("neighbourhood_", "") for s in c[18:-7]]
rty = [s.replace("room_type_", "") for s in c[-7:-3]]
loca = [s.replace("location_", "") for s in c[-3:]]

ngr = [s.replace(" ", "_") for s in ngr]
neb = [s.replace(" ", "_") for s in neb]
loca = [s.replace(" ", "_") for s in loca]
st.write(loca)
st.write(locd)

all_year_avail = False
low_avail = False
no_reviews = False
# button to make prediction

if st.button("Predict Price"):
    price = predict_price(latitude, longitude, minimum_nights, number_of_reviews,last_review, reviews_per_month, calculated_host_listings_count,availability_365, number_of_reviews_ltm, all_year_avail, low_avail,no_reviews,ngd[ngr[0]],ngd[ngr[1]],ngd[ngr[2]],ngd[ngr[3]],ngd[ngr[4]],ngd[ngr[5]],ned[neb[0]],ned[neb[1]],ned[neb[2]],ned[neb[3]],ned[neb[4]],ned[neb[5]],ned[neb[6]],ned[neb[7]],ned[neb[8]],ned[neb[9]],ned[neb[10]],ned[neb[11]],ned[neb[12]],ned[neb[13]],ned[neb[14]],ned[neb[15]],ned[neb[16]],ned[neb[17]],ned[neb[18]],ned[neb[19]],ned[neb[20]],ned[neb[21]],ned[neb[22]],ned[neb[23]],ned[neb[24]],ned[neb[25]],ned[neb[26]],ned[neb[27]],ned[neb[28]],ned[neb[29]],ned[neb[30]],ned[neb[31]],ned[neb[32]],ned[neb[33]],ned[neb[34]],ned[neb[35]],ned[neb[36]],ned[neb[37]],ned[neb[38]],ned[neb[39]],ned[neb[40]],ned[neb[41]],ned[neb[42]],ned[neb[43]],ned[neb[44]],ned[neb[45]],ned[neb[46]],ned[neb[47]],ned[neb[48]],ned[neb[49]],ned[neb[50]],ned[neb[51]],ned[neb[52]],ned[neb[53]],ned[neb[54]],ned[neb[55]],ned[neb[56]],ned[neb[57]],ned[neb[58]],ned[neb[59]],ned[neb[60]],ned[neb[61]],ned[neb[62]],ned[neb[63]],ned[neb[64]],ned[neb[65]],ned[neb[66]],ned[neb[67]],ned[neb[68]],ned[neb[69]],ned[neb[70]],ned[neb[71]],ned[neb[72]],ned[neb[73]],ned[neb[74]],ned[neb[75]],ned[neb[76]],ned[neb[77]],ned[neb[78]],ned[neb[79]],ned[neb[80]],ned[neb[81]],ned[neb[82]],ned[neb[83]],ned[neb[84]],ned[neb[85]],ned[neb[86]],ned[neb[87]],ned[neb[88]],ned[neb[89]],ned[neb[90]],ned[neb[91]],ned[neb[92]],ned[neb[93]],ned[neb[94]],ned[neb[95]],ned[neb[96]],ned[neb[97]],ned[neb[98]],ned[neb[99]],ned[neb[100]],ned[neb[101]],ned[neb[102]],ned[neb[103]],ned[neb[104]],ned[neb[105]],ned[neb[106]],ned[neb[107]],ned[neb[108]],ned[neb[109]],ned[neb[110]],ned[neb[111]],ned[neb[112]],ned[neb[113]],ned[neb[114]],ned[neb[115]],ned[neb[116]],ned[neb[117]],ned[neb[118]],ned[neb[119]],ned[neb[120]],ned[neb[121]],ned[neb[122]],ned[neb[123]],ned[neb[124]],ned[neb[125]],ned[neb[126]],ned[neb[127]],ned[neb[128]],ned[neb[129]],ned[neb[130]],ned[neb[131]],ned[neb[132]],ned[neb[133]],ned[neb[134]],ned[neb[135]],ned[neb[136]],ned[neb[137]],ned[neb[138]],ned[neb[139]],ned[neb[140]],ned[neb[141]],ned[neb[142]],ned[neb[143]],ned[neb[144]],ned[neb[145]],ned[neb[146]],ned[neb[147]],ned[neb[148]],ned[neb[149]],ned[neb[150]],ned[neb[151]],ned[neb[152]],ned[neb[153]],ned[neb[154]],ned[neb[155]],ned[neb[156]],ned[neb[157]],ned[neb[158]],ned[neb[159]],ned[neb[160]],ned[neb[161]],ned[neb[162]],ned[neb[163]],ned[neb[164]],ned[neb[165]],ned[neb[166]],ned[neb[167]],ned[neb[168]],ned[neb[169]],ned[neb[170]],ned[neb[171]],ned[neb[172]],ned[neb[173]],ned[neb[174]],ned[neb[175]],ned[neb[176]],ned[neb[177]],ned[neb[178]],ned[neb[179]],ned[neb[180]],ned[neb[181]],ned[neb[182]],ned[neb[183]],ned[neb[184]],ned[neb[185]],ned[neb[186]],ned[neb[187]],ned[neb[188]],ned[neb[189]],ned[neb[190]],ned[neb[191]],ned[neb[192]],ned[neb[193]],ned[neb[194]],ned[neb[195]],ned[neb[196]],ned[neb[197]],ned[neb[198]],ned[neb[199]],ned[neb[200]],ned[neb[201]],ned[neb[202]],ned[neb[203]],ned[neb[204]],ned[neb[205]],ned[neb[206]],ned[neb[207]],ned[neb[208]],ned[neb[209]],ned[neb[210]],ned[neb[211]],ned[neb[212]],ned[neb[213]],ned[neb[214]],ned[neb[215]],ned[neb[216]],ned[neb[217]],ned[neb[218]],ned[neb[219]],ned[neb[220]],ned[neb[221]],ned[neb[222]],ned[neb[223]],ned[neb[224]],ned[neb[225]],ned[neb[226]],ned[neb[227]],ned[neb[228]],ned[neb[229]],ned[neb[230]],ned[neb[231]],ned[neb[232]],ned[neb[233]],ned[neb[234]],ned[neb[235]],ned[neb[236]],ned[neb[237]],ned[neb[238]],ned[neb[239]],ned[neb[240]],ned[neb[241]],ned[neb[242]],ned[neb[243]],ned[neb[244]],ned[neb[245]],ned[neb[246]],ned[neb[247]],ned[neb[248]],ned[neb[249]],ned[neb[250]],ned[neb[251]],ned[neb[252]],ned[neb[253]],ned[neb[254]],ned[neb[255]],ned[neb[256]],ned[neb[257]],ned[neb[258]],ned[neb[259]],ned[neb[260]],ned[neb[261]],ned[neb[262]],ned[neb[263]],ned[neb[264]],ned[neb[265]],ned[neb[266]], rtd[rty[0]],rtd[rty[1]],rtd[rty[2]],rtd[rty[3]],locd[loca[0]],locd[loca[1]],locd[loca[2]])

    st.write("The predicted price for your rental is **$%.2f** per night." % price)


Overwriting app.py


In [155]:

!streamlit run app.py &>/content/logs.txt &

In [156]:
!npx localtunnel --port 8501

npx: installed 22 in 2.264s
your url is: https://tidy-hands-teach.loca.lt
^C


In [68]:
!pip install streamlit==1.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: streamlit
    Found existing installation: streamlit 1.22.0
    Uninstalling streamlit-1.22.0:
      Successfully uninstalled streamlit-1.22.0
